In [5]:
import os
from os.path import join
dir_path = os.path.dirname(os.path.realpath('__file__'))
ftir_dir = join(dir_path, 'ball-data', 'ftir')
felt_dir = join(ftir_dir, 'felt')
rubber_dir = join(ftir_dir, 'rubber')


/home/david/Repos/Balls-So-Hard/ball-data/ftir/rubber


In [ ]:
for line in open(file_path, 'r'):
    if word in line:
        print line